## Functional connectivity

This notebook extracts individual connectivity matrix of the resting-state networks in [@dosenbach2007] and [@difumo2018] atlases. The connectivity matrix, of shape $N_{\text{subjects}} \times N_{\text{regions}} \times N_{\text{regions}}$, will be stored as annotated NetCDF4 files in the following files:

- `data/julia2018_resting/connectivity_dosenbach2007.nc`.
- `data/julia2018_resting/connectivity_difumo_64_2.nc`.

## Setup

In [2]:
from typing import Tuple

from tqdm import tqdm

import numpy as np
import pandas as pd
import xarray as xr
import nilearn
from nilearn import connectome

Load the time-series of the cognitive control network:

In [3]:
DATASETS = {
  # 'dosenbach2007': xr.open_dataset('data/julia2018_resting/timeseries_dosenbach2007.nc'),
  # 'difumo_64_2': xr.open_dataset('data/julia2018_resting/timeseries_difumo_64_2.nc'),
  'difumo_128_2': xr.open_dataset('data/julia2018_resting/timeseries_difumo_128_2.nc')
}

## Fit connectivity matrices


In [4]:
# some subjects are missing resting time-series, so we need to filter them out.

def get_valid_timeseries(timeseries: xr.DataArray) -> Tuple[np.ndarray, list[np.ndarray]]:
  missing_mask = timeseries.isnull().all(dim=['region','timestep'])
  valid_timeseries = [ts.T for ts in timeseries.values[~missing_mask]]
  return missing_mask, valid_timeseries

timeseries = {
  key: get_valid_timeseries(ds['timeseries'])
  for key, ds in DATASETS.items()
}

In [5]:
# outputs
conn_kinds = ['covariance','correlation', 'partial correlation', 'tangent', 'precision']

for ds_name, (missing_mask, valid_timeseries) in timeseries.items():
  
  dataset = DATASETS[ds_name]
  dataset.attrs['description'] = f'Resting state connectivity matrices for {ds_name}'
  missing_indices = np.flatnonzero(missing_mask)
  
  for kind in tqdm(conn_kinds, desc=ds_name):
    
    # calculate connectivity measure
    cm = nilearn.connectome.ConnectivityMeasure(kind=kind, vectorize=False)
    conn = cm.fit_transform(valid_timeseries)
    
    # now refill the places of the missing time-series with nan values
    nan_insertion_indices = missing_indices - np.arange(missing_indices.shape[0])
    conn = np.insert(conn, nan_insertion_indices, np.full_like(conn[0], np.nan), axis=0)

    # DEBUG: just to make sure missing values are handled correctly
    assert np.equal(np.isnan(conn).all(axis=(1,2)), missing_mask).all()

    # add to the dataset
    ds_key_name = kind.replace(' ', '_') + '_connectivity'
    dataset[ds_key_name] = xr.DataArray(conn, dims=['subject', 'region', 'region'])

  # now store the dataset
  dataset.to_netcdf(f'data/julia2018_resting/connectivity_{ds_name}.nc', engine='netcdf4')
  print(f'{ds_name} connectivity saved successfully.')

difumo_128_2: 100%|██████████| 5/5 [00:01<00:00,  3.07it/s]

difumo_128_2 connectivity saved successfully.
